Teacher

In [1]:
import numpy as np
import os
from skimage import io
from skimage.transform import resize
import scipy.io as spio
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import sys
import torchvision.models as models
import torch.nn.functional as F
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from torch.utils.data.dataset import Subset
torch.manual_seed(1234) 

In [2]:
epochs = np.arange(1, 51)
glass_accuracy_test = []
gender_accuracy_test = []
testing_error_pose = []
glass_accuracy_train = []
gender_accuracy_train = []
training_error_pose = []
val_error_pose = []
glass_accuracy_val = []
gender_accuracy_val = []
best_val_loss = float('inf')
best_model_state = None
early_stopping_rounds = 5
patience = 0
NumEpochs = 50
torch.manual_seed(1234) 

num_classes_task1 = 2 
num_classes_task2 = 2 
num_classes_task3 = 3 

In [3]:
import torch
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True


In [4]:
img_size = [224, 224]
startFrom=0
# NumEpochs = 30
batch_size = 100
threadsTraining=1  
dataset_path='/kaggle/input/mtl-aflw-data'

In [5]:
imagesPath='/kaggle/input/mtl-aflw-data/cropped_resized_AFLW/images'
mat=spio.loadmat('/kaggle/input/mtl-aflw-data/cropped_resized_AFLW/matrices/dataAFLW.mat')
mat2=spio.loadmat('/kaggle/input/mtl-aflw-data/cropped_resized_AFLW/matrices/protocolAFLW.mat')

PresenceGender=mat2['TrainMask_Gender']
PresenceGlasses=mat2['TrainMask_Glasses']
PresencePose=mat2['TrainMask_Pose']
LabelPresenceMat=np.concatenate((PresenceGender, PresenceGlasses, PresencePose))
LabelPresenceMat=np.transpose(LabelPresenceMat)
TrainingSampleCount_taskWise=np.sum(LabelPresenceMat,axis=0)


In [6]:
elementsInTest = int(PresenceGender.shape[1]*0.3)
TrainingSampleCount = int(PresenceGender.shape[1]*0.7)

In [7]:
def loadCompleteDataset(startFrom, dataset_path,TrainingSampleCount):
    ArrayFaceID=mat['ArrayFaceID']
    ArrayGender_oneHot=mat['ArrayGender_oneHot']
    ArrayGlasses_oneHot=mat['ArrayGlasses_oneHot']
    ArraysPose=mat['ArraysPose']
    
    delMask=np.arange(startFrom,startFrom+elementsInTest)
    
    TestArrayFaceID = ArrayFaceID[startFrom:startFrom+elementsInTest:1,:]
    TrainArrayFaceID = ArrayFaceID
    TrainArrayFaceID = np.delete(TrainArrayFaceID,delMask,axis=0)
    TestStrNames = [str(i) for i in TestArrayFaceID]
    TestStrNames = [i[1:6] for i in TestStrNames]
    TrainStrNames = [str(i) for i in TrainArrayFaceID]
    TrainStrNames = [i[1:6] for i in TrainStrNames]
    
    TestArrayGender_oneHot = ArrayGender_oneHot[startFrom:startFrom+elementsInTest:1,:]
    TrainArrayGender_oneHot = ArrayGender_oneHot
    TrainArrayGender_oneHot = np.delete(TrainArrayGender_oneHot,delMask,axis=0)
    
    TestArrayGlasses_oneHot = ArrayGlasses_oneHot[startFrom:startFrom+elementsInTest:1,:]
    TrainArrayGlasses_oneHot = ArrayGlasses_oneHot
    TrainArrayGlasses_oneHot = np.delete(TrainArrayGlasses_oneHot,delMask,axis=0)
    
    TestArraysPose = ArraysPose[startFrom:startFrom+elementsInTest:1,:]
    TrainArraysPose = ArraysPose
    TrainArraysPose = np.delete(TrainArraysPose,delMask,axis=0)

    dataset_AFLW = datasetClass_AFLW(TrainStrNames, TrainArrayGender_oneHot, TrainArrayGlasses_oneHot, TrainArraysPose, LabelPresenceMat, imagesPath, TrainingSampleCount)
    dataset_AFLW_Test = datasetClass_AFLW_test(TestStrNames, TestArrayGender_oneHot, TestArrayGlasses_oneHot, TestArraysPose, imagesPath)
    return dataset_AFLW, TrainingSampleCount_taskWise.astype(float), dataset_AFLW_Test, elementsInTest


In [8]:
class datasetClass_AFLW(Dataset):
    def __init__(self, ArrayFaceID, ArrayGender_oneHot, ArrayGlasses_oneHot, ArraysPose, LabelPresenceMat, imagesPath, TrainingSampleCount):
        self.ArrayFaceID=ArrayFaceID
        self.ArrayGender_oneHot=torch.from_numpy(ArrayGender_oneHot)
        self.ArrayGlasses_oneHot=torch.from_numpy(ArrayGlasses_oneHot)
        self.ArraysPose=torch.from_numpy(ArraysPose)
        self.imagePath=imagesPath
        self.CountSamples = TrainingSampleCount#len(self.ArrayFaceID)
        self.LabelPresenceMat=LabelPresenceMat
    
    def __getitem__(self, index):
        imageName=self.ArrayFaceID[index]
        ## Original Clean Image [output]
        im_path = os.path.join(self.imagePath,imageName+'.png')
        img = io.imread(im_path)
        #following line normalizes image to [0, 1]
        img = resize(img, (img_size[0], img_size[1]))
        img = img.transpose(2, 0, 1)
        
        TaskwiseLabelPresence=self.LabelPresenceMat[index]
        
        label_gender = self.ArrayGender_oneHot[index]
        label_glass = self.ArrayGlasses_oneHot[index]
        label_pose = self.ArraysPose[index]
        return  img, label_gender, label_glass, label_pose, imageName, TaskwiseLabelPresence
        
    def __len__(self):
        return self.CountSamples


In [9]:
class datasetClass_AFLW_test(Dataset):
    def __init__(self, ArrayFaceID, ArrayGender_oneHot, ArrayGlasses_oneHot, ArraysPose, imagesPath):
        self.ArrayFaceID=ArrayFaceID
        self.ArrayGender_oneHot=torch.from_numpy(ArrayGender_oneHot)
        self.ArrayGlasses_oneHot=torch.from_numpy(ArrayGlasses_oneHot)
        self.ArraysPose=torch.from_numpy(ArraysPose)
        self.imagePath=imagesPath
        self.CountSamples = elementsInTest
    
    def __getitem__(self, index):
        imageName=self.ArrayFaceID[index]
        ## Original Clean Image [output]
        im_path = os.path.join(self.imagePath,imageName+'.png')
        img = io.imread(im_path)
        #following line normalizes image to [0, 1]
        img = resize(img, (img_size[0], img_size[1]))
        img = img.transpose(2, 0, 1)
        
        label_gender = self.ArrayGender_oneHot[index]
        label_glass = self.ArrayGlasses_oneHot[index]
        label_pose = self.ArraysPose[index]
        return img, label_gender, label_glass, label_pose, imageName
        
    def __len__(self):
        return self.CountSamples

    


In [10]:
torch.manual_seed(1234) 
dataset_AFLW, TrainingSampleCount_taskWise, dataset_AFLW_Test, elementsInTest = loadCompleteDataset(startFrom, dataset_path, TrainingSampleCount)

In [11]:
torch.manual_seed(1234) 
train_indices, val_indices = train_test_split(np.arange(len(dataset_AFLW)), test_size=0.3, random_state=42)
train_dataloader = DataLoader(Subset(dataset_AFLW, train_indices), batch_size=batch_size, shuffle=True, num_workers=threadsTraining, drop_last=True)
val_dataloader = DataLoader(Subset(dataset_AFLW, val_indices), batch_size=batch_size, shuffle=False, num_workers=threadsTraining, drop_last=True)
test_dataloader = DataLoader(dataset_AFLW_Test, shuffle=False, num_workers=threadsTraining, batch_size=batch_size,drop_last=True)

**TEACHER_MTL**

In [12]:
class SharedNetwork(nn.Module): 
    def __init__(self):
        super(SharedNetwork, self).__init__()
        resnet = models.resnet18(pretrained=True)
        self.shared_network = nn.Sequential(*list(resnet.children())[:-2])
        self.gap = nn.Sequential(nn.AdaptiveAvgPool2d(1))

    def forward(self, x):
        shared_embedding = self.gap(self.shared_network(x))
        return shared_embedding

class MultiTaskNetwork(nn.Module):
    def __init__(self, num_classes_task1, num_classes_task2, num_classes_task3):
        super(MultiTaskNetwork, self).__init__()
        self.shared_net = SharedNetwork().to(device)
        self.task1_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes_task1),
            nn.Softmax()
        ).to(device)
            
        self.task2_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes_task2),
            nn.Softmax()
        ).to(device)

        self.task3_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes_task3),
            nn.ReLU(inplace=True)
        ).to(device)

    def forward(self, x):
        shared_embedding = self.shared_net(x)
        task1_output = self.task1_head(shared_embedding.view(shared_embedding.size(0), -1))
        task2_output = self.task2_head(shared_embedding.view(shared_embedding.size(0), -1))
        task3_output = self.task3_head(shared_embedding.view(shared_embedding.size(0), -1))
        return task1_output, task2_output, task3_output

class myMSElossPose(torch.nn.Module):
    def __init__(self, batch_size):
        super(myMSElossPose, self).__init__()
        self.batch_size = batch_size

    def forward(self, predicted, target):
        BatchSampleCount=predicted.shape[0]
        error = torch.sub(predicted,target)
        squaredError = torch.mul(error, error)
        allSum = torch.sum(squaredError)
        FinalLoss = torch.div(allSum, 3*BatchSampleCount)
        return FinalLoss


In [13]:
def calculate_classification_metrics(labels, predictions):
    predictions_index = np.argmax(predictions, axis=1)
    labels_index = np.argmax(labels, axis=1)
    true_1 = []
    pred_1 = []
    for i in range(len(predictions)):
        true_1.append(labels[i][0])
        if(predictions[i][0] >=0.5):
            pred_1.append(1)
        else:
            pred_1.append(0)

    true_2 = []
    pred_2 = []
    for i in range(len(predictions)):
        true_2.append(labels[i][1])
        if(predictions[i][1] >=0.5):
            pred_2.append(1)
        else:
            pred_2.append(0)

    precision_1 = precision_score(true_1, pred_1)
    recall_1 = recall_score(true_1, pred_1)
    accuracy_1 = accuracy_score(true_1, pred_1)
    f1_1 = f1_score(true_1, pred_1)

    precision_2 = precision_score(true_2, pred_2)
    recall_2 = recall_score(true_2, pred_2)
    accuracy_2 = accuracy_score(true_2, pred_2)
    f1_2 = f1_score(true_2, pred_2)


    print('For Class 0:')
    print(f"Precision_0: {precision_1:.4f}, Recall_0: {recall_1:.4f}, Accuracy_0: {accuracy_1:.4f}, F1 Score_0: {f1_1:.4f}")
    print('For Class 1:')
    print(f"Precision_1: {precision_2:.4f}, Recall_1: {recall_2:.4f}, Accuracy_1: {accuracy_2:.4f}, F1 Score_1: {f1_2:.4f}")
    return accuracy_1


In [14]:
multi_task_net = MultiTaskNetwork(num_classes_task1, num_classes_task2, num_classes_task3).to(device)

for m in multi_task_net.modules():
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)


criterion_task1 = nn.BCELoss().to(device)
criterion_task2 = nn.BCELoss().to(device)
criterion_task3 = myMSElossPose(batch_size).to(device)


optimizer = optim.SGD(multi_task_net.parameters(), lr=0.001, momentum=0.9)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 134MB/s]


In [15]:
def training_result(epoch, multi_task_net, train_dataloader):
    criterion_task3 = myMSElossPose(batch_size).to(device)
    multi_task_net.eval()

    y_true_glass = []
    y_pred_glass = []
    y_true_gender = []
    y_pred_gender = []
    y_true_pose = []
    y_pred_pose = []
    R_te_errorPose=0
    te_countPose=0

    # Iterate over the test dataset
    with torch.no_grad():
        for i, data in enumerate(train_dataloader, 0):
            img, label_gender, label_glass, label_pose, imageName, TaskwiseLabelPresence = data
            img = img.to(device)
            label_gender = label_gender.to(device)
            label_glass = label_glass.to(device)
            label_pose = label_pose.to(device)
            TaskwiseLabelPresence = TaskwiseLabelPresence.numpy()
            img, label_gender, label_glass, label_pose = img.float(), label_gender.float(), label_glass.float(), label_pose.float()

            # Make predictions
            FC_output_gender, FC_output_glasses, FC_output_angle = multi_task_net(img)

            gender_IndSelect = np.where(TaskwiseLabelPresence[:, 0] == 1)
            glasses_IndSelect = np.where(TaskwiseLabelPresence[:, 1] == 1)
            pose_IndSelect = np.where(TaskwiseLabelPresence[:, 2] == 1)

            # Calculate task-wise metrics
            if gender_IndSelect[0].shape[0] != 0:
                y_true_gender.append(label_gender)
                y_pred_gender.append(FC_output_gender)

            if glasses_IndSelect[0].shape[0] != 0:
                y_true_glass.append(label_glass)
                y_pred_glass.append(FC_output_glasses)

            if pose_IndSelect[0].shape[0] != 0:
                y_true_pose.append(label_pose)
                y_pred_pose.append(FC_output_angle)
                loss3 = criterion_task3(FC_output_angle, label_pose)
                R_te_errorPose += loss3.item()
                te_countPose = te_countPose+1

   
    y_true_gender = np.concatenate([t.cpu().numpy() for t in y_true_gender], axis=0)
    y_pred_gender = np.concatenate([t.cpu().numpy() for t in y_pred_gender], axis=0)
    y_true_glass = np.concatenate([t.cpu().numpy() for t in y_true_glass], axis=0)
    y_pred_glass = np.concatenate([t.cpu().numpy() for t in y_pred_glass], axis=0)
    y_true_pose = np.concatenate([t.cpu().numpy() for t in y_true_pose], axis=0)
    y_pred_pose = np.concatenate([t.cpu().numpy() for t in y_pred_pose], axis=0)
    y_true_pose = y_true_pose.reshape(-1)
    y_pred_pose = y_pred_pose.reshape(-1)

    # Print the results
    r2 = r2_score(y_true_pose,y_pred_pose)
    training_error_pose.append(R_te_errorPose/te_countPose)

    print('No of epochs Trained: ',epoch+1)
    print('For Gender Task:')
    gender_accuracy_train.append(calculate_classification_metrics(y_true_gender,y_pred_gender))
    print('For Glasses Task:')
    glass_accuracy_train.append(calculate_classification_metrics(y_true_glass,y_pred_glass))
    print('For Pose Task:')       
    print(f"R2 Score: {r2:.4f}")
    print('Training error pose: ' + str(R_te_errorPose/te_countPose))
    multi_task_net.train()


In [16]:
def validation_result(epoch, multi_task_net, val_dataloader):
    criterion_task3 = myMSElossPose(batch_size).to(device)
    multi_task_net.eval()

    y_true_gender, y_pred_gender = [], []
    y_true_glass, y_pred_glass = [], []
    y_true_pose, y_pred_pose = [], []
    R_te_errorPose=0
    te_countPose=0


    with torch.no_grad():
        for i, data in enumerate(val_dataloader, 0):
            img, label_gender, label_glass, label_pose, imageName, TaskwiseLabelPresence = data
            img = img.to(device)
            label_gender = label_gender.to(device)
            label_glass = label_glass.to(device)
            label_pose = label_pose.to(device)
            TaskwiseLabelPresence = TaskwiseLabelPresence.numpy()
            img, label_gender, label_glass, label_pose = img.float(), label_gender.float(), label_glass.float(), label_pose.float()

            FC_output_gender, FC_output_glasses, FC_output_angle = multi_task_net(img)

            gender_IndSelect = np.where(TaskwiseLabelPresence[:, 0] == 1)
            glasses_IndSelect = np.where(TaskwiseLabelPresence[:, 1] == 1)
            pose_IndSelect = np.where(TaskwiseLabelPresence[:, 2] == 1)

            if gender_IndSelect[0].shape[0] != 0:
                y_true_gender.append(label_gender)
                y_pred_gender.append(FC_output_gender)

            if glasses_IndSelect[0].shape[0] != 0:
                y_true_glass.append(label_glass)
                y_pred_glass.append(FC_output_glasses)

            if pose_IndSelect[0].shape[0] != 0:
                y_true_pose.append(label_pose)
                y_pred_pose.append(FC_output_angle)
                loss3 = criterion_task3(FC_output_angle, label_pose)
                R_te_errorPose += loss3.item()
                te_countPose = te_countPose+1


    y_true_gender = np.concatenate([t.cpu().numpy() for t in y_true_gender], axis=0)
    y_pred_gender = np.concatenate([t.cpu().numpy() for t in y_pred_gender], axis=0)
    y_true_glass = np.concatenate([t.cpu().numpy() for t in y_true_glass], axis=0)
    y_pred_glass = np.concatenate([t.cpu().numpy() for t in y_pred_glass], axis=0)
    y_true_pose = np.concatenate([t.cpu().numpy() for t in y_true_pose], axis=0)
    y_pred_pose = np.concatenate([t.cpu().numpy() for t in y_pred_pose], axis=0)
    y_true_pose = y_true_pose.reshape(-1)
    y_pred_pose = y_pred_pose.reshape(-1)

    # Print the results
    r2 = r2_score(y_true_pose,y_pred_pose)
    val_error_pose.append(R_te_errorPose/te_countPose)

    print('No of epochs Trained: ',epoch+1)
    print('For Gender Task:')
    gender_accuracy_val.append(calculate_classification_metrics(y_true_gender,y_pred_gender))
    print('For Glasses Task:')
    glass_accuracy_val.append(calculate_classification_metrics(y_true_glass,y_pred_glass))
    print('For Pose Task:')       
    print(f"R2 Score: {r2:.4f}")
    print('Training error pose: ' + str(R_te_errorPose/te_countPose))
    multi_task_net.train()

In [17]:
def testing_result(epoch, multi_task_net, test_dataloader):
    
        criterion_task3 = myMSElossPose(batch_size).to(device)
        multi_task_net.eval()

        y_true_glass = []
        y_pred_glass = []
        y_true_gender = []
        y_pred_gender = []
        y_true_pose = []
        y_pred_pose = []
        R_te_errorPose=0
        te_countPose=0


        with torch.no_grad():
            for i, data in enumerate(test_dataloader, 0):
                img, label_gender, label_glass, label_pose, imageName = data
                img = img.to(device)
                label_gender = label_gender.to(device)
                label_glass = label_glass.to(device)
                label_pose = label_pose.to(device)
                img, label_gender, label_glass, label_pose = img.float(), label_gender.float(), label_glass.float(), label_pose.float()

                # Make predictions
                FC_output_gender, FC_output_glasses, FC_output_angle = multi_task_net(img)


                # Calculate task-wise metrics
                y_true_gender.append(label_gender)
                y_pred_gender.append(FC_output_gender)

                y_true_glass.append(label_glass)
                y_pred_glass.append(FC_output_glasses)

                y_true_pose.append(label_pose)
                y_pred_pose.append(FC_output_angle)
                loss3 = criterion_task3(FC_output_angle, label_pose)
                R_te_errorPose += loss3.item()
                te_countPose = te_countPose+1

        
        y_true_gender = np.concatenate([t.cpu().numpy() for t in y_true_gender], axis=0)
        y_pred_gender = np.concatenate([t.cpu().numpy() for t in y_pred_gender], axis=0)
        y_true_glass = np.concatenate([t.cpu().numpy() for t in y_true_glass], axis=0)
        y_pred_glass = np.concatenate([t.cpu().numpy() for t in y_pred_glass], axis=0)
        y_true_pose = np.concatenate([t.cpu().numpy() for t in y_true_pose], axis=0)
        y_pred_pose = np.concatenate([t.cpu().numpy() for t in y_pred_pose], axis=0)
        y_true_pose = y_true_pose.reshape(-1)
        y_pred_pose = y_pred_pose.reshape(-1)

        r2 = r2_score(y_true_pose,y_pred_pose)
        testing_error_pose.append(R_te_errorPose/te_countPose)

        # Print the results
        print('No of epochs Trained: ',epoch+1)
        print('For Gender Task:')
        gender_accuracy_test.append(calculate_classification_metrics(y_true_gender,y_pred_gender))
        print('For Glasses Task:')
        glass_accuracy_test.append(calculate_classification_metrics(y_true_glass,y_pred_glass))
        print('For Pose Task:')       
        print(f"R2 Score: {r2:.4f}")
        print('Testing error pose: ' + str(R_te_errorPose/te_countPose))
        multi_task_net.train()

In [18]:
for epoch in range(NumEpochs):
    print("Epoch: %d" % epoch)
    rloss1 = 0.0
    rloss2 = 0.0
    rloss3 = 0.0
    rnet_loss = 0.0

    multi_task_net.train()
    for i, data in enumerate(train_dataloader, 0):
        img, label_gender, label_glass, label_pose, imageName, TaskwiseLabelPresence = data
        img = img.to(device)
        label_gender = label_gender.to(device)
        label_glass = label_glass.to(device)
        label_pose = label_pose.to(device)
        TaskwiseLabelPresence = TaskwiseLabelPresence.numpy()
      
        img, label_gender, label_glass, label_pose = img.float(), label_gender.float(), label_glass.float(), label_pose.float()

        # Zero the gradients
        optimizer.zero_grad()

        FC_output_gender, FC_output_glasses, FC_output_angle = multi_task_net(img)

        gender_IndSelect = np.where(TaskwiseLabelPresence[:, 0] == 1)
        glasses_IndSelect = np.where(TaskwiseLabelPresence[:, 1] == 1)
        pose_IndSelect = np.where(TaskwiseLabelPresence[:, 2] == 1)

        if gender_IndSelect[0].shape[0] != 0:
            loss1 = criterion_task1(FC_output_gender[gender_IndSelect[0]], label_gender[gender_IndSelect[0]])
            rloss1 += loss1.item()
        else:
            loss1 = 0.0

        if glasses_IndSelect[0].shape[0] != 0:
            loss2 = criterion_task2(FC_output_glasses[glasses_IndSelect], label_glass[glasses_IndSelect])
            rloss2 += loss2.item()
        else:
            loss2 = 0.0

        if pose_IndSelect[0].shape[0] != 0:
            loss3 = criterion_task3(FC_output_angle[pose_IndSelect[0]], label_pose[pose_IndSelect[0]])
            rloss3 += loss3.item()
        else:
            loss3 = 0.0

        NetLoss = loss1 + loss2 + loss3

        if epoch != 0:
            NetLoss.backward()
            optimizer.step()

        rnet_loss += NetLoss.item()

    print("Task1 Loss: {:.4f}, Task2 Loss: {:.4f}, Task3 Loss: {:.4f}, Total Loss: {:.4f}".format(
        rloss1 / len(train_dataloader),
        rloss2 / len(train_dataloader),
        rloss3 / len(train_dataloader),
        rnet_loss / len(train_dataloader)
    ))


    multi_task_net.eval()
    with torch.no_grad():
        val_loss = 0.0
        for i, data in enumerate(val_dataloader, 0):
            img, label_gender, label_glass, label_pose, imageName, TaskwiseLabelPresence = data
            img = img.to(device)
            label_gender = label_gender.to(device)
            label_glass = label_glass.to(device)
            label_pose = label_pose.to(device)
            TaskwiseLabelPresence = TaskwiseLabelPresence.numpy()
            img, label_gender, label_glass, label_pose = img.float(), label_gender.float(), label_glass.float(), label_pose.float()

            FC_output_gender, FC_output_glasses, FC_output_angle = multi_task_net(img)

            gender_IndSelect = np.where(TaskwiseLabelPresence[:, 0] == 1)
            glasses_IndSelect = np.where(TaskwiseLabelPresence[:, 1] == 1)
            pose_IndSelect = np.where(TaskwiseLabelPresence[:, 2] == 1)

            loss1 = criterion_task1(FC_output_gender[gender_IndSelect[0]], label_gender[gender_IndSelect[0]]) if gender_IndSelect[0].shape[0] != 0 else 0.0
            loss2 = criterion_task2(FC_output_glasses[glasses_IndSelect], label_glass[glasses_IndSelect]) if glasses_IndSelect[0].shape[0] != 0 else 0.0
            loss3 = criterion_task3(FC_output_angle[pose_IndSelect[0]], label_pose[pose_IndSelect[0]]) if pose_IndSelect[0].shape[0] != 0 else 0.0


            val_loss += (loss1 + loss2 + loss3).item()


    val_loss /= len(val_dataloader)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = multi_task_net.state_dict()
        patience = 0
    else:
        patience += 1
        if patience >= early_stopping_rounds:
            print("Early stopping triggered.")
            break


    print('Training Result')
    training_result(epoch,multi_task_net, train_dataloader)
    print('Validation Result')
    validation_result(epoch,multi_task_net, val_dataloader)
    print('Testing Result')
    testing_result(epoch,multi_task_net, test_dataloader)


    multi_task_net.load_state_dict(best_model_state)
    torch.save(multi_task_net, 'Teacher_MTL.pth')

Epoch: 0


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Task1 Loss: 0.7286, Task2 Loss: 0.3507, Task3 Loss: 0.1670, Total Loss: 1.2463
Training Result
No of epochs Trained:  1
For Gender Task:
For Class 0:
Precision_0: 0.5750, Recall_0: 0.9542, Accuracy_0: 0.5689, F1 Score_0: 0.7176
For Class 1:
Precision_1: 0.4481, Recall_1: 0.0500, Accuracy_1: 0.5689, F1 Score_1: 0.0900
For Glasses Task:
For Class 0:
Precision_0: 0.9288, Recall_0: 1.0000, Accuracy_0: 0.9288, F1 Score_0: 0.9631
For Class 1:
Precision_1: 0.0000, Recall_1: 0.0000, Accuracy_1: 0.9288, F1 Score_1: 0.0000
For Pose Task:
R2 Score: -18.1603
Training error pose: 0.13548252022410012
Validation Result


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


No of epochs Trained:  1
For Gender Task:
For Class 0:
Precision_0: 0.5843, Recall_0: 0.9479, Accuracy_0: 0.5774, F1 Score_0: 0.7230
For Class 1:
Precision_1: 0.4619, Recall_1: 0.0622, Accuracy_1: 0.5774, F1 Score_1: 0.1096
For Glasses Task:
For Class 0:
Precision_0: 0.9229, Recall_0: 1.0000, Accuracy_0: 0.9229, F1 Score_0: 0.9599
For Class 1:
Precision_1: 0.0000, Recall_1: 0.0000, Accuracy_1: 0.9229, F1 Score_1: 0.0000
For Pose Task:
R2 Score: -17.7745
Training error pose: 0.1328635750072343
Testing Result


...

No of epochs Trained:  5
For Gender Task:
For Class 0:
Precision_0: 0.0000, Recall_0: 0.0000, Accuracy_0: 0.3818, F1 Score_0: 0.0000
For Class 1:
Precision_1: 0.3818, Recall_1: 1.0000, Accuracy_1: 0.3818, F1 Score_1: 0.5526
For Glasses Task:
For Class 0:
Precision_0: 0.9220, Recall_0: 1.0000, Accuracy_0: 0.9220, F1 Score_0: 0.9594
For Class 1:
Precision_1: 0.0000, Recall_1: 0.0000, Accuracy_1: 0.9220, F1 Score_1: 0.0000
For Pose Task:
R2 Score: -3.5729
Testing error pose: 0.026452805755623414


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 5


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Task1 Loss: 58.1878, Task2 Loss: 0.2639, Task3 Loss: 0.0237, Total Loss: 58.4755
Early stopping triggered.


In [19]:
df_train = pd.DataFrame({'Gender_Accuracy_Train': gender_accuracy_train,
                  'Glass_Accuracy_Train': glass_accuracy_train,
                  'Pose_error': training_error_pose})

df_val =  pd.DataFrame({'Gender_Accuracy_Val': gender_accuracy_val,
                  'Glass_Accuracy_Val': glass_accuracy_val,
                  'Pose_error_val': val_error_pose})

df_test = pd.DataFrame({'Gender_Accuracy_test': gender_accuracy_test,
                  'Glass_Accuracy_test': glass_accuracy_test,
                  'Pose_error_test': testing_error_pose})

df_train.to_excel('train.xlsx',index=False)
df_val.to_excel('val.xlsx',index=False)
df_test.to_excel('test.xlsx',index=False)